In [1]:
# Source
datadownloadpath_Source = './ATLAS_R2.0/'
datasavepath_Source = './Brain_lesion_dataset/'

In [100]:
import glob
import tqdm
import SimpleITK as sitk
import numpy as np
import pandas as pd
from collections import Counter
from scipy.ndimage import label, binary_fill_holes
import os
import shutil

# Source Date Preprocessing

In [61]:
# Image example
# ./ATLAS_R2.0/ATLAS_2/Training/R001/sub-r001s001/ses-1/anat
# sub-r001s001_ses-1_space-MNI152NLin2009aSym_T1w.nii.gz
# Label example
# ./ATLAS_R2.0/ATLAS_2/Training/R001/sub-r001s001/ses-1/anat/
# sub-r001s001_ses-1_space-MNI152NLin2009aSym_label-L_desc-T1lesion_mask.nii.gz

imgs = glob.glob(datadownloadpath_Source + "ATLAS_2/Training/*/*/*/*/*T1w.nii.gz")
imgs = [ fid for fid in sorted(imgs) ]
segs = glob.glob(datadownloadpath_Source + "ATLAS_2/Training/*/*/*/*/*mask.nii.gz")
segs = [ fid for fid in sorted(segs) ]

pids = [pid.split("/")[-1].split(".")[0].split("_")[0] for pid in imgs]

In [80]:
imgs

['./ATLAS_R2.0/ATLAS_2/Training/R001/sub-r001s001/ses-1/anat/sub-r001s001_ses-1_space-MNI152NLin2009aSym_T1w.nii.gz',
 './ATLAS_R2.0/ATLAS_2/Training/R001/sub-r001s002/ses-1/anat/sub-r001s002_ses-1_space-MNI152NLin2009aSym_T1w.nii.gz',
 './ATLAS_R2.0/ATLAS_2/Training/R001/sub-r001s003/ses-1/anat/sub-r001s003_ses-1_space-MNI152NLin2009aSym_T1w.nii.gz',
 './ATLAS_R2.0/ATLAS_2/Training/R001/sub-r001s004/ses-1/anat/sub-r001s004_ses-1_space-MNI152NLin2009aSym_T1w.nii.gz',
 './ATLAS_R2.0/ATLAS_2/Training/R001/sub-r001s005/ses-1/anat/sub-r001s005_ses-1_space-MNI152NLin2009aSym_T1w.nii.gz',
 './ATLAS_R2.0/ATLAS_2/Training/R001/sub-r001s006/ses-1/anat/sub-r001s006_ses-1_space-MNI152NLin2009aSym_T1w.nii.gz',
 './ATLAS_R2.0/ATLAS_2/Training/R001/sub-r001s007/ses-1/anat/sub-r001s007_ses-1_space-MNI152NLin2009aSym_T1w.nii.gz',
 './ATLAS_R2.0/ATLAS_2/Training/R001/sub-r001s008/ses-1/anat/sub-r001s008_ses-1_space-MNI152NLin2009aSym_T1w.nii.gz',
 './ATLAS_R2.0/ATLAS_2/Training/R001/sub-r001s009/ses-1/

In [81]:
segs

['./ATLAS_R2.0/ATLAS_2/Training/R001/sub-r001s001/ses-1/anat/sub-r001s001_ses-1_space-MNI152NLin2009aSym_label-L_desc-T1lesion_mask.nii.gz',
 './ATLAS_R2.0/ATLAS_2/Training/R001/sub-r001s002/ses-1/anat/sub-r001s002_ses-1_space-MNI152NLin2009aSym_label-L_desc-T1lesion_mask.nii.gz',
 './ATLAS_R2.0/ATLAS_2/Training/R001/sub-r001s003/ses-1/anat/sub-r001s003_ses-1_space-MNI152NLin2009aSym_label-L_desc-T1lesion_mask.nii.gz',
 './ATLAS_R2.0/ATLAS_2/Training/R001/sub-r001s004/ses-1/anat/sub-r001s004_ses-1_space-MNI152NLin2009aSym_label-L_desc-T1lesion_mask.nii.gz',
 './ATLAS_R2.0/ATLAS_2/Training/R001/sub-r001s005/ses-1/anat/sub-r001s005_ses-1_space-MNI152NLin2009aSym_label-L_desc-T1lesion_mask.nii.gz',
 './ATLAS_R2.0/ATLAS_2/Training/R001/sub-r001s006/ses-1/anat/sub-r001s006_ses-1_space-MNI152NLin2009aSym_label-L_desc-T1lesion_mask.nii.gz',
 './ATLAS_R2.0/ATLAS_2/Training/R001/sub-r001s007/ses-1/anat/sub-r001s007_ses-1_space-MNI152NLin2009aSym_label-L_desc-T1lesion_mask.nii.gz',
 './ATLAS_R2.

In [64]:
pids

['sub-r001s001',
 'sub-r001s002',
 'sub-r001s003',
 'sub-r001s004',
 'sub-r001s005',
 'sub-r001s006',
 'sub-r001s007',
 'sub-r001s008',
 'sub-r001s009',
 'sub-r001s010',
 'sub-r001s011',
 'sub-r001s012',
 'sub-r001s013',
 'sub-r001s014',
 'sub-r001s015',
 'sub-r001s016',
 'sub-r001s017',
 'sub-r001s018',
 'sub-r001s019',
 'sub-r001s020',
 'sub-r001s021',
 'sub-r001s022',
 'sub-r001s023',
 'sub-r001s024',
 'sub-r001s025',
 'sub-r001s026',
 'sub-r001s027',
 'sub-r001s028',
 'sub-r001s029',
 'sub-r001s030',
 'sub-r001s031',
 'sub-r001s032',
 'sub-r001s033',
 'sub-r001s034',
 'sub-r001s036',
 'sub-r001s037',
 'sub-r001s038',
 'sub-r001s039',
 'sub-r002s001',
 'sub-r002s002',
 'sub-r002s003',
 'sub-r002s004',
 'sub-r002s005',
 'sub-r002s006',
 'sub-r002s007',
 'sub-r002s008',
 'sub-r002s009',
 'sub-r002s010',
 'sub-r002s011',
 'sub-r002s012',
 'sub-r003s001',
 'sub-r003s002',
 'sub-r003s003',
 'sub-r003s004',
 'sub-r003s005',
 'sub-r003s006',
 'sub-r003s007',
 'sub-r003s008',
 'sub-r003s009

In [65]:
metainfo = pd.read_excel(datadownloadpath_Source + "ATLAS_2/20220425_ATLAS_2.0_MetaData.xlsx")

In [71]:
allscan = Counter(metainfo['Scanner Brand'])

In [72]:
allscan

Counter({'GE Signa Excite': 38,
         'Siemens Trio': 88,
         'Siemens Prisma': 70,
         'GE 750 Discovery': 137,
         'GE Signa HD-X': 29,
         'Siemens TrioTim': 42,
         'Siemens Verio': 15,
         'Philips': 18,
         'Philips Achieva': 78,
         'Siemens Allegra': 49,
         'Siemens Sonata': 2,
         'Siemens Vision': 45,
         'Siemens Magnetom Skyra': 25,
         'Siemens Skyra': 19})

In [79]:
metainfo[metainfo['Subject ID'] == pids[0]]['Scanner Brand'][0]

'GE Signa Excite'

## Step 1, Intensity Normalization

In [85]:
IMG_FOLDER_SAVE = datasavepath_Source + 'reorganized/'
if os.path.isdir(IMG_FOLDER_SAVE) == False:
    os.makedirs(IMG_FOLDER_SAVE)

In [102]:
image_savename = 'image.nii.gz'
seg_savename = 'seg.nii.gz'
for (imgfile, segfile, pid) in zip(imgs, segs, pids):
    img_obj = sitk.ReadImage( imgfile )
    
    array = sitk.GetArrayFromImage(img_obj)

    pixel_mean = np.mean(array)
    pixel_std = np.std(array)
    array = (array - pixel_mean) / pixel_std

    normalized_img = sitk.GetImageFromArray(array)
    normalized_img.CopyInformation(img_obj)
    
    scaninfo = metainfo[metainfo['Subject ID'] == pid]['Scanner Brand'].tolist()[0]
    
    scanfolder = IMG_FOLDER_SAVE + scaninfo
    if os.path.isdir(scanfolder) == False:
        os.makedirs(scanfolder)
    
    casefolder = scanfolder + '/' + pid
    if os.path.isdir(casefolder) == False:
        os.makedirs(casefolder)
    
    sitk.WriteImage(normalized_img, casefolder + '/' + image_savename, True) 
    
    shutil.copyfile(segfile, casefolder + '/' + seg_savename)
    
    print('Id ' + pid + ' processed')

Id sub-r001s001 processed
Id sub-r001s002 processed
Id sub-r001s003 processed
Id sub-r001s004 processed
Id sub-r001s005 processed
Id sub-r001s006 processed
Id sub-r001s007 processed
Id sub-r001s008 processed
Id sub-r001s009 processed
Id sub-r001s010 processed
Id sub-r001s011 processed
Id sub-r001s012 processed
Id sub-r001s013 processed
Id sub-r001s014 processed
Id sub-r001s015 processed
Id sub-r001s016 processed
Id sub-r001s017 processed
Id sub-r001s018 processed
Id sub-r001s019 processed
Id sub-r001s020 processed
Id sub-r001s021 processed
Id sub-r001s022 processed
Id sub-r001s023 processed
Id sub-r001s024 processed
Id sub-r001s025 processed
Id sub-r001s026 processed
Id sub-r001s027 processed
Id sub-r001s028 processed
Id sub-r001s029 processed
Id sub-r001s030 processed
Id sub-r001s031 processed
Id sub-r001s032 processed
Id sub-r001s033 processed
Id sub-r001s034 processed
Id sub-r001s036 processed
Id sub-r001s037 processed
Id sub-r001s038 processed
Id sub-r001s039 processed
Id sub-r002s

Id sub-r018s012 processed
Id sub-r019s001 processed
Id sub-r019s004 processed
Id sub-r019s007 processed
Id sub-r019s008 processed
Id sub-r019s009 processed
Id sub-r019s010 processed
Id sub-r019s012 processed
Id sub-r023s001 processed
Id sub-r023s002 processed
Id sub-r023s003 processed
Id sub-r023s007 processed
Id sub-r023s008 processed
Id sub-r023s009 processed
Id sub-r023s014 processed
Id sub-r023s017 processed
Id sub-r024s002 processed
Id sub-r024s003 processed
Id sub-r024s005 processed
Id sub-r024s008 processed
Id sub-r024s011 processed
Id sub-r024s012 processed
Id sub-r024s013 processed
Id sub-r024s014 processed
Id sub-r024s018 processed
Id sub-r024s019 processed
Id sub-r024s021 processed
Id sub-r027s001 processed
Id sub-r027s006 processed
Id sub-r027s009 processed
Id sub-r027s013 processed
Id sub-r027s015 processed
Id sub-r027s017 processed
Id sub-r027s023 processed
Id sub-r027s031 processed
Id sub-r027s032 processed
Id sub-r027s035 processed
Id sub-r027s041 processed
Id sub-r027s

Id sub-r050s003 processed
Id sub-r050s005 processed
Id sub-r050s006 processed
Id sub-r050s008 processed
Id sub-r050s009 processed
Id sub-r050s015 processed
Id sub-r052s001 processed
Id sub-r052s002 processed
Id sub-r052s007 processed
Id sub-r052s010 processed
Id sub-r052s011 processed
Id sub-r052s014 processed
Id sub-r052s015 processed
Id sub-r052s016 processed
Id sub-r052s019 processed
Id sub-r052s021 processed
Id sub-r052s023 processed
Id sub-r052s025 processed
Id sub-r052s026 processed
Id sub-r052s027 processed
Id sub-r052s029 processed
Id sub-r052s031 processed
Id sub-r052s032 processed
